In [ ]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve


In [ ]:
#website_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
retail_data = pd.read_excel('OnlineRetail.xlsx') # This may take a couple minutes

In [ ]:
retail_data.head()

In [ ]:
retail_data.info()

In [ ]:
cleaned_retail = retail_data.loc[pd.isnull(retail_data.CustomerID) == False]

In [ ]:
cleaned_retail.info()

In [ ]:
item_lookup = cleaned_retail[['StockCode', 'Description']].drop_duplicates() # Only get unique item/description pairs
item_lookup['StockCode'] = item_lookup.StockCode.astype(str) # Encode as strings for future lookup ease

In [ ]:
item_lookup.head()

In [ ]:
cleaned_retail.loc['CustomerID'] = cleaned_retail.CustomerID.astype(int) # Convert to int for customer ID

In [ ]:
cleaned_retail.query('CustomerID==13672')

In [ ]:
grouped_cleaned = cleaned_retail.groupby(['CustomerID', 'StockCode']).sum().reset_index() # Group together

In [ ]:
#we will use this later
xgboost_data  = cleaned_retail.groupby(['CustomerID', 'StockCode','Country']).sum().reset_index() # Group together
xgboost_data.loc[xgboost_data.Quantity==0,'Quantity'] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
xgboost_data = xgboost_data.query('Quantity > 0') # Only get customers where purchase totals were positive

In [ ]:
xgboost_data.head()

In [ ]:
print(xgboost_data.query('CustomerID==13672'))

In [ ]:
grouped_cleaned.head()

In [ ]:
grouped_cleaned.loc[grouped_cleaned.Quantity==0,'Quantity'] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased

In [ ]:
grouped_purchased = grouped_cleaned.query('Quantity > 0') # Only get customers where purchase totals were positive

In [ ]:
grouped_purchased.head()

In [ ]:
grouped_cleaned.head()

In [ ]:
grouped_purchased = grouped_purchased[['StockCode', 'Quantity', 'CustomerID']] # Get rid of unnecessary info

In [ ]:
grouped_purchased.query('CustomerID==13672')

In [ ]:
customers = list(np.sort(grouped_purchased.CustomerID.unique())) # Get our unique customers
products = list(grouped_purchased.StockCode.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.Quantity) # All of our purchases

rows = grouped_purchased.CustomerID.astype('category', categories = customers).cat.codes 
# Get the associated row indices
cols = grouped_purchased.StockCode.astype('category', categories = products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [ ]:
purchases_sparse

In [ ]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

In [ ]:
import random

In [ ]:
def make_train(purchases, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    purchases - the original purchases matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = purchases.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = purchases.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [ ]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

In [ ]:
np.shape(product_users_altered)

In [ ]:
import implicit
from implicit import alternating_least_squares

In [ ]:
%env OPENBLAS_NUM_THREADS=1

alpha = 15
#user_vecs, item_vecs = alternating_least_squares((product_train*alpha).astype('double'), 
                                                          #factors=20, 
                                                          #regularization = 0.1, 
                                                         #iterations = 50)

#self.user_factors[userid]
            
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=20)

#sparse matrix of item, user with alpha multiplication
item_user_data = (product_train*alpha).astype('double').T
print("item_user_data")
print(np.shape(item_user_data))

# train the model on a sparse matrix of item/user/confidence weights
model.fit(item_user_data)


# recommend items for a user
userid = 1
user_items = item_user_data.T.tocsr()
recommendations = model.recommend(userid, user_items)
user_vecs = model.user_factors
item_vecs = model.item_factors

In [ ]:
from sklearn import metrics

In [ ]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [ ]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [ ]:
np.shape(product_users_altered)

In [ ]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

In [ ]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [ ]:
products_arr

In [ ]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[item_lookup.StockCode.isin(prod_codes)]

In [ ]:
customers_arr[:5]

In [ ]:
get_items_purchased(12346, product_train, customers_arr, products_arr, item_lookup)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.Description.loc[item_lookup.StockCode == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'StockCode': codes, 'Description': descriptions}) # Create a dataframe 
    return final_frame[['StockCode', 'Description']] # Switch order of columns around


In [ ]:
rec_items(12346, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

In [ ]:
get_items_purchased(12353, product_train, customers_arr, products_arr, item_lookup)

In [ ]:
rec_items(12353, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

In [ ]:
get_items_purchased(12361, product_train, customers_arr, products_arr, item_lookup)

In [ ]:
rec_items(12361, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

In [ ]:
def prediction_matrix(mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup):
    '''
    This function will return the normalized prediction for each item and each user:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
  
    returns:
    
    - The full matrix of predictions for all users and items
    '''
    #item_factors_matrix = (item_vecs * np.shape(user_vecs)[0]).T  
    item_factors_matrix = item_vecs.T  
    rec_matrix = user_vecs.dot(item_factors_matrix) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    min_max.fit(rec_matrix)
    rec_matrix_scaled = min_max.transform(rec_matrix)
    return rec_matrix_scaled

In [ ]:
output_predictions = prediction_matrix(product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup)

## XGBOOST Ensemble ##

In [ ]:
print(xgboost_data.query('CustomerID==13672'))

In [ ]:
xgboost_data.info()

In [ ]:
#there is something wrong with this user
#xgboost_data = xgboost_data.query('CustomerID!=15802')

In [ ]:
#TODO: Turn country into a numerical variable, remove other columns from retail_data except unit price, add a column for ALS value, add column for target, set all to 1. Introduce ALS value from result matrix. Add random negatives by replicating every row, changing to a random user ID and getting their ALS  value for that item (ideally we would change item, but we'd need to know the unit price, which is not immediate)


In [ ]:
xgboost_data.head()

In [ ]:
#Let's add a column for the value of ALS, and one for the positive negative label
xgboost_data = xgboost_data.assign(ALS=0., label=1)

In [ ]:
xgboost_data.head()

In [ ]:
np.shape(xgboost_data)

In [ ]:
#now let's prepare a matrix of random negatives
xgboost_negatives = xgboost_data.copy()
random_list = np.random.choice(products_arr,266733)
xgboost_negatives['StockCode'] = random_list
xgboost_negatives['label'] =0

In [ ]:
xgboost_negatives.head()

In [ ]:
xgboost_data.head()

In [ ]:
frames = [xgboost_data,xgboost_negatives]
xgboost_final_data = pd.concat(frames)

In [ ]:
xgboost_final_data.head()

In [ ]:
xgboost_final_data.tail()

In [ ]:
users_index = pd.DataFrame({'User':customers_arr,'Pos':np.arange(customers_arr.size)}, index=customers_arr)
items_index = pd.DataFrame({'Item':products_arr,'Pos':np.arange(products_arr.size)}, index=products_arr)

In [ ]:
def set_ALS_values_fast(data,customers,products,predictions):
    x=np.array(customers.Pos[data.CustomerID]).astype(int)
    y=np.array(products.Pos[data.StockCode.astype(str)]).astype(int)
    data.ALS = predictions[x,y]    

In [ ]:
customers_inxgboostdata = np.array(users_index.Pos[xgboost_data.CustomerID]).astype(int)
items_inxgboostdata = np.array(items_index.Pos[xgboost_data.StockCode]).astype(int)

In [ ]:
items_index.min()

In [ ]:
set_ALS_values_fast(xgboost_final_data, users_index, items_index, output_predictions)

In [ ]:
xgboost_final_data.tail()

In [ ]:
np.shape(xgboost_final_data)

In [ ]:
xgboost_final_data.to_csv('Xgboostdata_with_neagatives.csv')

In [ ]:
xgboost_final_data.info()

We need to convert all features to numerical. We will use categorical encoding

In [ ]:
for feature in xgboost_final_data.columns: # Loop through all columns in the dataframe
    if xgboost_final_data[feature].dtype == 'object': # Only apply for columns with categorical strings
        xgboost_final_data[feature] = pd.Categorical(xgboost_final_data[feature]).codes # Replace strings with an integer

In [ ]:
xgboost_final_data.info()

In [ ]:
xgboost_final_data.tail()

We will now split into train and test

In [ ]:
#first we need to shuffle rows since we have now all positive and negatives together
xgboost_final_data = xgboost_final_data.sample(frac=1).reset_index(drop=True)
xgboost_final_data = xgboost_final_data.sample(frac=1).reset_index(drop=True)

In [ ]:
xgboost_final_data.tail()

In [ ]:
xgboost_final_data.head()

In [ ]:
#now we can select 20% of the data for testing
train_samples = int(len(xgboost_final_data)*0.8)
print (train_samples)
xgboost_train = xgboost_final_data[:train_samples].copy() 
xgboost_test = xgboost_final_data[train_samples:].copy()


In [ ]:
#We also need to separate target variable label from the datasets
label_train = xgboost_train.pop('label')
label_test = xgboost_test.pop('label')

In [ ]:
xgboost_train.info()

In [ ]:
xgboost_test.info()

In [ ]:
xgboost_test.head()

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [ ]:
xgboost_train.to_csv('xgboost_train.csv')

In [ ]:
label_train.to_csv('label_train.csv')

In [ ]:
#Look for more info on the XGBoost parameters here: https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}

ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
            'objective': 'binary:logistic'}
optimized_XGB = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'roc_auc', cv = 5, n_jobs = 1, verbose=2) 
#We optimize for AUC to compare with previous results

In [ ]:
#train XGB
optimized_XGB.fit(xgboost_train, label_train,eval_metric='auc')

In [ ]:
#train error
optimized_XGB.grid_scores_

In [ ]:
#we take our best model given cross validation results
best_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth':5, 'min_child_weight':1} 

train_xgb_matrix = xgb.DMatrix(xgboost_train, label_train) # Create our DMatrix to make XGBoost more efficient
best_model = xgb.train(best_params, train_xgb_matrix, num_boost_round = 432)

In [ ]:
#now let's check error on test set
test_xgb_matrix = xgb.DMatrix(xgboost_test)
from sklearn.metrics import roc_auc_score
label_pred = best_model.predict(test_xgb_matrix)
label_pred

In [ ]:
roc_auc_score(label_test, label_pred)

# Bonus Follow ups

* Improve overall auc by adding more features to xgboost ensemble
* Fine-tune xgboost ensemble by optimizing other hyperparameters (e.g. learning rate)
* Analyze how hyperparameters behave to metrics different from AUC
* The resulting AUC is "too good to be true". Can you figure out where there might be some data-leak happening?
